In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import glob

import tifffile as tf

import os

path = "/home/patrick/github/data"

filenames = glob.glob(os.path.join(path, "dm*/**/*.tif*"), recursive=True)

print(len(filenames))

In [ ]:
# filter lamella, landing

print(len(filenames))
# filenames = [fname for fname in filenames if "ref_landing" not in fname and "ref_lamella" not in fname and "trench" not in fname and "low_res" not in fname]

filenames = [fname for fname in filenames if "needle" in fname]
print(len(filenames))

In [ ]:
for fname in filenames[:10]:
    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
save_path = "/home/patrick/github/data/training"
for i, fname in enumerate(filenames):
    
    img = tf.imread(fname)
    new_fname = os.path.join(save_path,f"a{i:05d}.tif")

    tf.imsave(new_fname, img)


In [ ]:
train_filenames = sorted(glob.glob(os.path.join(save_path, "*.tif")))
for fname in train_filenames[:10]:

    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
import zarr

images = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "images", "*.tif*"), aszarr=True)) 
labels = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "labels","*.tif*"), aszarr=True))


print(images.shape)
print(labels.shape)


for img, label in zip(images, labels):


    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(label)
    plt.show()



### Model Testing

In [ ]:
%load_ext autoreload
%autoreload 2 


import glob
import os
import tifffile as tf
import matplotlib.pyplot as plt

images = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images", "*.tif*")))
labels = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/labels", "*.tif*")))

for ii, (i, l) in enumerate(zip(images, labels)):
    print(os.path.basename(i), os.path.basename(l))

    img, lbl = tf.imread(i), tf.imread(l)

    fig, ax = plt.subplots(1, 2, figsize=(7, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")
    ax[1].imshow(lbl)
    ax[1].set_title("Label")
    plt.show()
    
    if ii == 2:
        break



In [ ]:
filenames = sorted(glob.glob(os.path.join("/home/patrick/github/data/liftout/training/train/images/", "*.tif*")))

print("files: ", len(filenames))
# filenames.append(*list(sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images/", "*.tif*")))
print("files: ", len(filenames))
import random
random.shuffle(filenames)


from fibsem.segmentation.model import SegmentationModel

baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model.pt"
# checkpoint_2 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_01_51_n08_model.pt"
# checkpoint_5 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_00_58_n05_model.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_02_27_n10_model.pt"
# checkpoints = [baseline, checkpoint_2, checkpoint_5, checkpoint_10]

small_model = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_00_10_n10_model.pt"
# checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_09_22_n15_model.pt"
# checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_18_32_n20_model.pt"

checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_37_32_n10_model.pt"
checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_46_48_n15_model.pt"
checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_56_00_n20_model.pt"


checkpoints = [baseline, small_model, checkpoint_10, checkpoint_15, checkpoint_20]
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    print(i, fname) 

    fig, ax = plt.subplots(1, len(checkpoints) + 1, figsize=(15, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")


    titles = ["Baseline", "Update", "Checkpoint 10", "Checkpoint 15", "Checkpoint 20"]
    for j, (checkpoint, title) in enumerate(zip(checkpoints, titles), 1):
        model = SegmentationModel(checkpoint=checkpoint, num_classes=3)

        mask = model.inference(img)

        ax[j].imshow(mask)
        ax[j].set_title(title)

    save_path = "results34"
    os.makedirs(save_path, exist_ok=True)
    fig.savefig(os.path.join(save_path, f"{os.path.basename(fname).replace('tif', 'png')}"))

    plt.show()

### Pattern Testing


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, patterning
import matplotlib.pyplot as plt

# protocol_path = "/home/patrick/github/autoliftout/liftout/protocol/protocol.yaml"
protocol_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path)

In [ ]:
patterns_fn = {f"{pattern.name}": pattern for pattern in patterning.__PATTERNS__}

from pprint import pprint

pprint(patterns_fn)

In [ ]:
from fibsem.structures import FibsemPatternSettings, FibsemPattern

rect_pattern = patterns_fn["Rectangle"]()

rp = rect_pattern.define(protocol={"pattern": "Rectangle", "width": 10e-6, "height": 5e-6, 
                                              "depth": 2e-6, "rotation": 0, 
                                              "centre_x": 0, "centre_y": 0, 
                                              "scan_direction": "TopToBottom", "cleaning_cross_section": True})


print(rp)

In [ ]:



pattern_fn = patterns_fn["Trench"]()
print(pattern_fn.required_keys)
pattern = pattern_fn.define(protocol=settings.protocol["polish_lamella"])

pprint(pattern)

print("---------------------------------------")

In [ ]:
for p in patterns_fn:
    print(p, patterns_fn[p], patterns_fn[p]())

### Test AutoFOCUS



In [ ]:
%load_ext autoreload
%autoreload 2

import sys
from fibsem import utils, calibration, patterning
from fibsem.structures import BeamType

In [ ]:
microscope, settings = utils.setup_session()

In [ ]:
# default
calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON)

In [ ]:
wd = microscope.get("working_distance", BeamType.ELECTRON)
print(f"working distance: {wd:2e}")

In [ ]:
# sharpness
from fibsem.calibration import _sharpness, _dog
calibration.auto_focus_beam(microscope=microscope, settings=settings, 
                            beam_type=BeamType.ELECTRON, 
                            metric_fn=_sharpness, kwargs={"disk_size": 5}, 
                            num_steps=20, step_size=0.05e-3)

In [ ]:
calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON, metric_fn=_dog, num_steps=10, verbose=True)

In [ ]:
wd = wd
wd1 = microscope.get("working_distance", BeamType.ELECTRON)

In [ ]:
print(f"working distance: {wd:2e}") # default
print(f"working distance: {wd1:2e}") # fibsem 

In [ ]:
settings.image.beam_type = BeamType.ELECTRON
settings.image.save = True

from fibsem import acquire

microscope.set("working_distance", wd, BeamType.ELECTRON)
settings.image.label = "default_autofocus"
eb_image = acquire.new_image(microscope, settings.image)

In [ ]:
microscope.set("working_distance", wd1, BeamType.ELECTRON)
settings.image.label = "fibsem_autofocus"
eb_image = acquire.new_image(microscope, settings.image)

In [ ]:
import numpy as np
wds = np.linspace(3.5e-3, 4.5e-3, 20)

settings.image.save = True
settings.image.save_path = r"C:\Users\Admin\Github\fibsem\demo_2023-03-13-10-27-17AM"

for wd in wds:
    print(f"wd_{wd:.4e}".replace('.', '_'))
    microscope.set("working_distance", wd, BeamType.ELECTRON)
    settings.image.label = f"wd_{wd:.4e}".replace('.', '_')
    eb_image = acquire.new_image(microscope, settings.image)

## Manipulator Testing

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, movement

from fibsem.structures import BeamType, FibsemManipulatorPosition


microscope, settings = utils.setup_session()

In [ ]:
# microscope.insert_manipulator()
settings.image.hfw = 400e-6
images = acquire.take_reference_images(microscope, settings.image)
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(images[0].data, cmap="gray")
ax[0].set_title("Electron Image 01")
ax[1].imshow(images[1].data, cmap="gray")
ax[1].set_title("IOn Image 02")
plt.show()

# #position = FibsemManipulatorPosition(x=20e-6, y=20e-6, z=20e-6, r=0, t=0)
# microscope.move_manipulator_corrected(dx = 20e-6, dy = 20e-6, beam_type=BeamType.ELECTRON)
position = FibsemManipulatorPosition(z=20e-6)
microscope.move_manipulator_to_position_offset(position, name="EUCENTRIC")

images_new = acquire.take_reference_images(microscope, settings.image)
fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(images_new[0].data, cmap="gray")
ax[0].set_title("Electron Image 01")
ax[1].imshow(images_new[1].data, cmap="gray")
ax[1].set_title("IOn Image 02")
plt.show()
microscope.get_manipulator_position()


## Gas Injection System


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.structures import BeamType

from fibsem import utils, acquire, movement

microscope, settings = utils.setup_session()


In [ ]:
from fibsem import gis

print(gis.gis_protocol)

In [ ]:
gis.sputter_platinum(microscope, gis.gis_protocol)

In [ ]:
gis.cryo_sputter(microscope, gis.gis_protocol, name="cryo_sputter_eb_grid_01")


## Manipulator Position Testing



In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.structures import BeamType, FibsemManipulatorPosition, FibsemStagePosition

from liftout import actions
import matplotlib.pyplot as plt



microscope, settings = utils.setup_session(manufacturer="Thermo", ip_address="10.0.0.1")




In [ ]:
microscope.retract_manipulator()

settings.image.hfw = 400e-6

# eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)
# fig, ax = plt.subplots(1, 2, figsize=(10, 7))
# ax[0].imshow(eb_image.data, cmap="gray")
# ax[1].imshow(ib_image.data, cmap="gray")
# plt.show()

actions.move_needle_to_liftout_position(microscope)

eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()

actions.move_needle_to_landing_position(microscope)

eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()

actions.move_needle_to_reset_position(microscope)

eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(eb_image.data, cmap="gray")
ax[1].imshow(ib_image.data, cmap="gray")
plt.show()



### Tile Image Collection

- Collect tiled images
- Stitch Together
- Update metadata
- Click to Move, Save Positions from Image



In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.structures import BeamType, FibsemManipulatorPosition, FibsemStagePosition

import matplotlib.pyplot as plt
import os
import glob

from fibsem.imaging import _tile

microscope, settings = utils.setup_session(manufacturer="Thermo", ip_address="10.0.0.1")




In [ ]:
start_state = microscope.get_current_microscope_state()



In [ ]:
print(start_state.eb_settings.working_distance)
print(microscope.get_current_microscope_state().eb_settings.working_distance)

In [ ]:
microscope.set_microscope_state(start_state)

In [ ]:
import logging
PATH = os.path.join(os.getcwd(), "tile-images")
os.makedirs(PATH, exist_ok=True)


GRID_SIZE = 400e-6
TILE_SIZE = 50e-6
BEAM_TYPE = BeamType.ELECTRON
RESOLUTION = [1024, 1024]


settings.image.hfw = TILE_SIZE
settings.image.beam_type = BEAM_TYPE
settings.image.save = True
settings.image.save_path = PATH
settings.image.resolution = RESOLUTION
settings.image.dwell_time = 1e-6
settings.image.autocontrast = False

ddict = _tile._tile_image_collection(microscope, settings, GRID_SIZE, TILE_SIZE)
images = ddict["images"]
big_image = ddict["big_image"]

In [ ]:
 # stitch a composite image from 2d array of images




image = _tile._stitch_images(images=images, ddict=ddict, overlap=0)

# plot stitched and big image
fig, ax = plt.subplots(1, 2, figsize=(30, 15))
ax[0].imshow(image.data, cmap="gray")
ax[0].set_title("Stitched Image")
ax[1].imshow(big_image.data, cmap="gray")
ax[1].set_title("Big Image")
plt.show()



# TODO: more advanced version where we read the metadata to get the relative position of each image in space




In [ ]:

from fibsem import config as cfg
PATH = os.path.join(cfg.DATA_PATH, "tile")
from fibsem.structures import FibsemImage

image = FibsemImage.load(os.path.join(PATH, "stitched_image.tif"))

print(image.metadata.microscope_state.absolute_position)

###  Intersecting Circles

In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.structures import BeamType, FibsemManipulatorPosition, FibsemStagePosition

import matplotlib.pyplot as plt
import numpy as np





In [ ]:
# create random circles on image

import cv2
image = np.zeros((1024, 1024))
image2 = np.zeros((1024, 1024))

r = 50
for i in range(25):
    x = np.random.randint(0, 1024)
    y = np.random.randint(0, 1024)
    
    image = cv2.circle(image, (x, y), r, (1, 1, 1), -1)
    
    x = np.random.randint(0, 1024)
    y = np.random.randint(0, 1024)
    
    image2 = cv2.circle(image2, (x, y), r, (1, 1, 1), -1)



print(image.shape)


fig, ax = plt.subplots(1, 3, figsize=(10, 7))
ax[0].imshow(image, cmap="Greens")
ax[0].axis("off")
ax[1].imshow(image2, cmap="Reds")
ax[1].axis("off")
ax[2].imshow(image, cmap="Greens", alpha=0.5)
ax[2].imshow(image2, cmap="Reds", alpha=0.5)
ax[2].axis("off")
plt.show()


# detect where circles intersect

from skimage import measure

intersection = image + image2
intersection[intersection < 2] = 0
intersection[intersection == 2] = 1


# extract contour from intersection

contours = measure.find_contours(intersection, 0.8)

# plot intersection and contour

fig, ax = plt.subplots(1, 2, figsize=(10, 7))
ax[0].imshow(intersection, cmap="gray")
ax[0].axis("off")
ax[1].imshow(image, cmap="Greens", alpha=0.5)
ax[1].imshow(image2, cmap="Reds", alpha=0.5)
ax[1].axis("off")
for contour in contours:
    ax[1].plot(contour[:, 1], contour[:, 0], linewidth=2)
plt.show()





In [ ]:
mask = intersection
idxs = np.unique(mask)

for idx in idxs:
    if idx==0:
        continue

    # create a new image
    feature_mask = np.zeros_like(mask)
    feature_mask[mask==idx] = 1

    # detect features
    feature.detect(image, feature_mask)
    features.append(deepcopy(feature))

In [ ]:


%load_ext autoreload
%autoreload 2

from fibsem.segmentation.model import SegmentationModel, load_model
from fibsem.imaging import _tile

import matplotlib.pyplot as plt
import numpy as np


image = np.zeros(shape=(1024*4, 1024*4))
image.shape

model = load_model("/home/patrick/github/fibsem/fibsem/segmentation/models/model4.pt", "resnet34", 3)


In [ ]:
out = model.inference(image)



In [ ]:
plt.imshow(out)
plt.show()

In [ ]:

# cut image into tiles and predict on each tile


n_rows, n_cols = 4, 4
tile_size = 1024
overlap = 0
tiles = _tile._create_tiles(image, n_rows, n_cols, tile_size, overlap=0)


# reshape 
print(tiles.shape)


In [ ]:
out = model.inference(tiles, rgb=False)
print(out.shape)

# reshape output 
out = out.reshape(n_rows, n_cols, tile_size, tile_size)

# re-stitch together?
from fibsem.imaging import _tile
arr = _tile._stitch_arr(out)

# plot stitched and big image
fig, ax = plt.subplots(1, 2, figsize=(30, 15))
ax[0].imshow(arr, cmap="gray")
ax[0].set_title("Stitched Image")
ax[1].imshow(image, cmap="gray")
ax[1].set_title("Big Image")
plt.show()

## Point Reprojection



In [ ]:
from fibsem import utils, acquire
from fibsem.structures import FibsemImage, BeamType, FibsemStagePosition, Point

from fibsem.imaging import _tile

import matplotlib.pyplot as plt
import numpy as np


In [ ]:
microscope, settings = utils.setup_session()

In [ ]:
positions = []

In [ ]:
PATH = r"C:\Users\Admin\Github\fibsem\fibsem\log\data\tile\stitched-image-electron.tif"

image = FibsemImage.load(PATH)




# microscope._safe_absolute_stage_movement(image.metadata.microscope_state.absolute_position)
current_position = microscope.get_stage_position()
current_point = _calculate_repojection(image, current_position)
positions.append(current_position)
print("---------------------------------------")
base_position = image.metadata.microscope_state.absolute_position
base_point = _calculate_repojection(image, base_position)


# plot on matplotlib
fig = plt.figure(figsize=(15, 15))
plt.imshow(image.data, cmap="gray")


COLOURS = ["r+", "g+", "b+", "c+", "m+", "y+"]
points = _reproject_positions(image, positions)
# plt.plot(current_point.x, current_point.y, "r+", ms=100, markeredgewidth=5, label="current_position")
for i, pt in enumerate(points):
    plt.plot(pt.x, pt.y, COLOURS[i], ms=100, markeredgewidth=5, label=f"position_{i}")

plt.plot(base_point.x, base_point.y, "w+", ms=100, markeredgewidth=5 , label="base position")
plt.legend(loc="best")
plt.show()

In [ ]:
positions.pop(-1)

In [ ]:
points

for pt in points:
    # reverse to list
    pt = pt.y, pt.x
    print(pt)

In [ ]:
from pprint import pprint
pprint(positions + [current_position])



In [ ]:
%load_ext autoreload
%autoreload 2
# draw positions on image

from fibsem import utils
from fibsem.imaging import _tile
from fibsem.structures import Point, FibsemStagePosition, FibsemImage
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np

PATH = r"C:\Users\Admin\Github\fibsem\fibsem\log\data\tile\stitched-image-ion.tif"
POS_PATH = r"C:\Users\Admin\Github\fibsem\fibsem\log\data\tile\stitched-image-ion-positions.yaml"


image = FibsemImage.load(PATH)

pdict = utils.load_yaml(POS_PATH)
positions = [FibsemStagePosition.__from_dict__(p) for p in pdict]

fig = _tile._plot_positions(image, positions, show=False)

plt.show()

